In [1]:
# Import Libraries 
import hakom
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import getpass
import urllib.request, json 
import pandas as pd
from datetime import datetime, timedelta
#teständerung

## Initialize Python Connector

In [2]:
api_pwd = getpass.getpass()


In [ ]:
auth_params = hakom.authorization.AuthParams(
    username = 'aeneas',
    password = api_pwd,
)

service_config = hakom.service.ServiceConfig(
    base_url="https://fehostedwebapp.azurewebsites.net/MAIN/api/",
    auth_params=auth_params,
    repository='ZAMS'
)

hakom.initialize(service_config)


## Define Functions

In [4]:
def parse_inmet(station_str):
    """ Takes inmet station number as input to download data from beginning of 2018 to end 2022. 
    Returns all data in a pandas-df.
    Expexted shape is (43824, 27)  """
    data =[]
    for year in range(2018,2023):
        date_begin = '{}-01-01'.format(year)
        date_end = '{}-12-31'.format(year)
        url_str = 'https://apitempo.inmet.gov.br/estacao/{}/{}/{}'.format(date_begin, date_end, str(station_str))
        print(url_str)
      
        with urllib.request.urlopen(url_str) as url:
            temp = pd.read_json(url.read().decode())
        data.append(temp)
    data = pd.concat(data)
    return data


def timeframe_inmet(data):
    """Takes pandas-df, adds timestamp as index (hour). 
    Returns whole data frame or only the temperature column """
    data['timestamp'] = pd.to_datetime(data['DT_MEDICAO']) + pd.to_timedelta(data['HR_MEDICAO'] // 100, unit='h')
    data = data.set_index('timestamp')
    data = data.asfreq('h')
    
    df_temp = data[['TEM_INS']]
    df_wdir = data[['VEN_DIR']]
    df_wspeed = data[['VEN_VEL']]
    return data, df_temp


def inmet_TEMP_into_TSM_format(df):
    """Takes a temperature df and transforms it into HAKOM data format.
    Returns HAKOM data format, ready to load into DB """
    df = hakom.pandas_interop.create_timestamp_colum_from_index(df)
    df['flag'] = 9
    one_hour_interval = hakom.types.Interval(hakom.types.Intervals.Hour)
    unit_label = '°C'
    tsdc = hakom.pandas_interop.to_hakom_tsdata_collection(
        df.dropna(),
        interval=one_hour_interval,
        unit=unit_label,
        dt_flag_value_order=[2,1,3]
    )
    return tsdc

## Go for the loop